### 구글 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 패키지 설치

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from pickle import dump

## 데이터 불러오기

In [ ]:
path = '/content/drive/MyDrive/AIVLE/BIGP/data/'
normal_data = pd.read_csv(path + 'press_data_normal.csv', index_col=0)
outlier_data = pd.read_csv(path + 'outlier_data.csv', index_col=0)

# 데이터 전처리

TimeStamp 속성을 datetime타입으로

In [ ]:
normal_data['TimeStamp'] = pd.to_datetime(normal_data['TimeStamp'])
outlier_data['TimeStamp'] = pd.to_datetime(outlier_data['TimeStamp'])

### 중복 TimeStamp row 제거

In [ ]:
normal_data['TimeStamp'].duplicated().sum()

np.int64(1)

In [ ]:
normal_data = normal_data.groupby('TimeStamp').mean().reset_index()

### 진폭으로 변환

In [ ]:
col = ['AI0_Vibration', 'AI1_Vibration', 'AI2_Current']

In [ ]:
normal_data[col] = normal_data[col].map(lambda x: abs(x))
outlier_data[col] = outlier_data[col].map(lambda x: abs(x))

## 데이터 분할

X-Y 분할

In [ ]:
# X값으로 AI0_Vibration, AI1_Vibration, AI2_Current 사용
X_normal = normal_data[col]
y_normal = normal_data['Equipment_state']
X_outlier = outlier_data[col]
y_outlier = outlier_data['Equipment_state']

Train-Test-Valid 분할

In [ ]:
X_train_normal = X_normal[:15000]
y_train_normal = y_normal[:15000]
X_test_normal = X_normal[15000:]
y_test_normal = y_normal[15000:]
X_test_anomaly = X_outlier
y_test_anomaly = y_outlier

## 최대 최소 정규화

In [ ]:
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train_normal)
X_test_normal_scaled = scaler.transform(X_test_normal)
X_test_anomaly_scaled = scaler.transform(X_test_anomaly)

y_train_normal = np.array(y_train_normal)
y_test_normal = np.array(y_test_normal)
y_test_anomaly = np.array(y_test_anomaly)

스케일러 저장

In [ ]:
dump(scaler, open(path+'press_fault_scaler.pkl', 'wb'))

## LSTM 학습을 위한 shape 변경
LSTM셀은 3차원 데이터를 입력받으므로, 데이터의 구조를 변경할 필요가 있음.

- 데이터의 구조는 (n, sequence, m)으로, n은 시계열 데이터의 길이, sequence는 모델이 다음 시점의 미래 예측을 위해 사용하는 과거 데이터의 길이, m은 변수의 개수를 나타낸다.
- 본 데이터셋에서는 모델이 과거 20개(2초)의 데이터를 사용하여 100시점(10초) 이후의 이상작동을 탐지할 수 있게 구성하였다.
- 학습을 위해 list type으로 편집된 데이터를 numpy array타입으로 변환한다.

In [ ]:
sequence = 20
X_train, Y_train = [], []
for i in range(len(X_train_scaled) - sequence-100):
    X_train.append(X_train_scaled[i:i+sequence])
    Y_train.append(y_train_normal[i+sequence+ 100])

X_test_normal, Y_test_normal = [], []
for i in range(len(X_test_normal_scaled)- sequence-100):
    X_test_normal.append(X_test_normal_scaled[i:i+sequence])
    Y_test_normal.append(y_test_normal[i+sequence+ 100])

X_test_anomaly, Y_test_anomaly = [], []
for i in range(len(X_test_anomaly_scaled)- sequence-100):
    X_test_anomaly.append(X_test_anomaly_scaled[i:i+sequence])
    Y_test_anomaly.append(y_test_anomaly[i+sequence+ 100])

X_test_normal = np.array(X_test_normal)
Y_test_normal = np.array(Y_test_normal)
X_test_anomaly = np.array(X_test_anomaly)
Y_test_anomaly = np.array(Y_test_anomaly)

In [ ]:
X_test_normal.shape

(4879, 20, 3)

## 임곗값 지정을 위한 검증 데이터셋 구성

In [ ]:
X_valid_normal, Y_valid_normal = X_test_normal[:880, :, :], Y_test_normal[:880]
X_test_normal, Y_test_normal = X_test_normal[880:, :, :], Y_test_normal[880:]
X_valid_anomaly, Y_valid_anomaly = X_test_anomaly[:300, :, :], Y_test_anomaly[:300]
X_test_anomaly, Y_test_anomaly = X_test_anomaly[300:, :, :], Y_test_anomaly[300:]

X_valid = np.vstack((X_valid_normal, X_valid_anomaly))
Y_valid = np.hstack((Y_valid_normal, Y_valid_anomaly))

X_test = np.vstack((X_test_normal, X_test_anomaly))
Y_test = np.hstack((Y_test_normal, Y_test_anomaly))

In [ ]:
# 데이터셋 구성 확인
X_train, Y_train = np.array(X_train), np.array(Y_train)
X_valid, Y_valid = np.array(X_valid), np.array(Y_valid)
X_test, Y_test = np.array(X_test), np.array(Y_test)

print('X_train:', X_train.shape, 'Y_train:', Y_train.shape)
print('X_valid:', X_valid.shape, 'Y_valid:', Y_valid.shape)
print('X_test:', X_test.shape, 'Y_test:', Y_test.shape)

X_train: (14880, 20, 3) Y_train: (14880,)
X_valid: (1180, 20, 3) Y_valid: (1180,)
X_test: (4179, 20, 3) Y_test: (4179,)


분할한 데이터셋 저장

In [ ]:
import os
os.makedirs(path+'set/', exist_ok=True)

np.save(path+'set/X_train.npy', X_train)
np.save(path+'set/Y_train.npy', Y_train)
np.save(path+'set/X_valid.npy', X_valid)
np.save(path+'set/Y_valid.npy', Y_valid)
np.save(path+'set/X_test.npy', X_test)
np.save(path+'set/Y_test.npy', Y_test)